In [ ]:
# Initial Setting (font, display)
!sudo apt-get install -y fonts-nanum # 글씨체 다운로드
!sudo fc-cache -fv # 폰트 리스트 갱신
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
import matplotlib
get_ipython().run_line_magic('config', "InlineBackend.figure_format='retina'") # 화질 보정

import os
import re
import pandas as pd
import numpy as np
import seaborn as sns

# NLP packages
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
print('Ready')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 0s (45.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:
!pip3 install kss
!pip3 install git+https://github.com/haven-jeon/PyKoSpacing.git # 한글 띄어쓰기(PyKoSpacing)
!pip install git+https://github.com/ssut/py-hanspell.git # 한글 맞춤법 검사기(Py-hanspell)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1) 잡코리아/링커리어 웹 크롤링
- 페이지수가 url 중 page=부분에서 변경됨
- 한 페이지당 20개의 합격자기소개서

In [ ]:
!pip3 install selenium

In [ ]:
# Packages for data crawling
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import time
import sys
import requests

# colab에서 selenium을 사용하기 위해서는 지정해줘야 하는 부분
!apt-get update
!apt install chromium-chromedriver

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("disable-gpu")

options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
driver = webdriver.Chrome('chromedriver', options=options)

delay_time = 1
driver.implicitly_wait(delay_time)

In [ ]:
# 게시글 데이터 담을 데이터프레임 생성
linkcareer = pd.DataFrame(columns = ['year', 'company', 'position', 'content'])

# 한 페이지당 게시글 20개, 1~90페이지 크롤링 (91, 40)
for k in range(260, 351): 
  for i in range(1, 40, 2): # 홀수 단위로 변경됨
    try:
      url = "https://linkareer.com/cover-letter/search?page=%d&tab=all" % k
      driver.get(url)
      info = driver.find_elements_by_xpath("/html/body/div[1]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[%d]" % i)
          
      for n in info:
        try:
          year = n.find_element_by_xpath("/html/body/div[1]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[%d]/div[1]/a/div/p[1]" % i)
          year = year.text
        except:
          year = "NULL"
        try:
          company = n.find_element_by_xpath("/html/body/div[1]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[%d]/div[1]/a/div/p[2]" % i)
          company = company.text
        except:
          company = "NULL"
        try:
          position = n.find_element_by_xpath("/html/body/div[1]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[%d]/div[1]/a/div/p[3]" % i)
          position = position.text
        except:
          position = "NULL"

        # 각 자기소개서 본문으로 접속
        n.click()
        linkcareer_post = []
        try:
          content = driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[2]/div/div/div[2]/div[2]/div[3]/article")
          content = content.text
          linkcareer_post.append(content)
        except:
          content = "NULL"
          linkcareer_post.append(content)
      
      new_data = {'year': year, 'company': company, 'position': position, 'content': content}
      linkcareer = linkcareer.append(new_data, ignore_index=True)
      print('pass')
    except:
      print('error')

In [ ]:
# 상위 데이터 5개 확인
pd.set_option('display.max_colwidth', -1) # truncated되지 않고 모두 보여주기
linkcareer[:5]

### 2) 데이터 전처리
- 빈값 NULL로 처리해서 제거
- 정규표현식으로 한글/영문만 남겨두기
- 띄어쓰기 교정 (pykospacing 패키지 활용)
- 기업명 불용어 처리 (모든 기업명을 1개의 데이터프레임으로 만든 다음 불용어로 처리해서 제거)
- 한 줄에 하나의 문장이 들어가도록 형태 변환

In [ ]:
linkcareer_1 = pd.read_csv('/content/drive/MyDrive/linkcareer_crawling_1.csv')
linkcareer_2 = pd.read_csv('/content/drive/MyDrive/linkcareer_crawling_2.csv')
del linkcareer_1['Unnamed: 0']
del linkcareer_2['Unnamed: 0']

In [ ]:
# 2개의 링커리어 df 이어붙이기
linkcareer = pd.concat([linkcareer_1, linkcareer_2], axis=0)
pd.set_option('display.max_colwidth', -1) #truncated되지 않고 post 모두 보여주기
linkcareer[:1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


,year,company,position,content
0,2021 상반기,현대엔지니어링,자산관리 자산설비,"본인이 회사를 선택할때의 기준은 무엇이며, 왜 현대엔지니어링이 그 기준에 적합한지를 기술해 주십시오.(800~1200) - 건설기술연구원에서 지열원 히트펌프 COP을 올리기 위해 삼 주 정도 기계실에서 상주한 적이 있습니다. 시설관리자와 함께 설비를 유지 보수하면서 책임감을 느끼고 자신의 설비에 대한 문제를 해결하는 것에 매력을 느꼈고 건물 설비 관리에 관심을 두었습니다.\n\n회사란 단순히 돈을 버는 수단이 아닌 회사 성장과 함께 나 자신 자아실천의 장이 되어야 한다고 생각합니다.\n현대엔지니어링은 완성차, 철강 공장 등 산업시설과 상업빌딩, 의료시설 등 일반건축에 이르기까지 넓은 사업영역을 가지고 있을 뿐만 아니라 해외에서는 개발도상국의 의료시설, 산업시설뿐만 아니라 동남아시아, 동유럽 등에 수요가 예상되며 해외에 넓은 사업영역을 가지고 있습니다.\n저는 정적인 자산설비 엔지니어가 되기보단 넓은 영역에서 각각의 설비에 능통하고 최첨단 설비에 적용하여 배우며 폭넓고 높은 지식을 갖고 넓은 분야에서 성과를 올리는 동적인 자산설비 엔지니어가 되고 싶습니다.\n\n저는 세 가지 강점이 있습니다\n첫째, 자산설비 엔지니어는 자신이 담당한 설비에 대한 책임감이 있어야 한다고 생각합니다. 저는 BMTKOREA 현장실습을 할 때, 스티어링펌프개조공사중 한 챔버에 달린 두 개의 유압펌프 압력 차로 인해 오일이 누출이 발생하였고 더 큰 피해를 예방하고자 오일을 뒤집어쓰며 손으로 고정해 사장님에게 책임감을 인정받으며 칭찬받은 경험이 있습니다 \n둘째, 설비를 혼자 관리하는 것이 아닌 여러 인원이 관리하기 때문에 원활한 의사소통을 통해 공통적인 목표 지향이 필요합니다. 저는 인도공과대학교 연구실 인턴을 할 때 자성유체 droplet 프로젝트를 통해 현지 적응 못 하는 팀원을 위해 노력한 적이 있습니다. 매일 두 번을 같이 써브웨이를 먹었고, 매일 저녁에 같이 현지인들과 축구를 하였습니다. 이를 통해 저는 의사소통을 위해 상대방의 상태에 대한 인식도 중요하다고 배웠습니다. \n셋째, 수많은 종류의 설비와 최첨단 설비에 대응하기 위해서는 입사 후에도 배움의 자세가 필요하다고 생각합니다. 건설기술연구원에서 BEMS를 위한 데이터 수집업무를 주로 하면서 부족함을 많이 느꼈고 실습이 끝나고도 BEMS 협회에서 하는 전문가양성프로젝트에 참여하여 지식을 쌓을 수 있었습니다.\n\n이러한 강점을 바탕으로 자산설비 엔지니어가 되어 설비를 유지, 보수하고 또한 설비상의 문제를 해결함으로써 고객에게 쾌적한 환경을 제공할 것이고 건물 소유고객의 자산가치를 높이기 위해 노력할 것입니다.\n\n\n현대엔지니어링 직무분야에 지원하게 된 이유와 선택직무에 본인이 적합하다고 판단할 수 있는 이유 및 근거를 제시해 주십시오(800~1200) - 자산설비 기술 엔지니어로서 설비 고장 분석 및 개선을 위해 데이터에 대한 통찰력과 설비 문제 \n파악 능력이 필요하다고 생각합니다. 다음과 같은 경험으로 필요한 역량들을 기를 수 있었습니다.\n\n[분석이 바탕 된 전력소비량 감소]\n3학년 여름방학 때 건설기술연구원 건물에너지부서에서 현장실습을 하여 전력소비량 분석을 통해 전체 소비 전력량을 낮춘 경험이 있습니다. \n전력 데이터 시각화 중 19년 5월부터 구조동 전력량의 기울기 증가를 확인하였습니다. 18년~19년 7월까지 월별로 로우 데이터를 통해 평균 전력량을 비교하였고 17시 이후 불필요하게 가동된 항온항습기 때문이란 것을 알았습니다. \n제어 시 전력 이득을 예측하여 관리자에게 중요성을 일깨웠고 올바른 스케줄제어가 되어 결국 3달 후 전체 소비 전력량 21% 감소에 일조하였습니다.\n\n[설비 부품 교체를 통한 COP 향상]\n설비 효율 유지를 위한 BEMS 모니터링 중 기계실 지열원 히트펌프의 효율이 낮아진 것을 발견하였습니다.\n원인 분석을 위해 관련 논문 4편을 통해 유량, 입 출구 온도 차 등 주요 파라미터를 확인하여 18년도 데이터와 비교 분석하였습니다. 하지만 오차 범위의 정확한 인지가 어려워 문제점에 대한 방향성을 못 잡았고 이를 매뉴얼을 통해 이상적인 공급온도 13도, 환수온도 15~17도에 못 미쳐 펌프 자체에 낮은 효율이 발생하여 생긴 것을 파악했습니다. \n사수와 기계실의 설비를 직접 체크하였고 압축기가 노후화되어 냉매가 누설되고 있었던 것을 확인했습니다. Leak 이외에도 장비 수명을 고려하여 압축기를 교체하였고, 19년도 11월에 지열 COP을 4.5까지 올려 효율의 향상을 이루어 냈습니다.\n\n이를 통해 데이터 분석 결과 활용의 중요성을 몸소 깨달았고 설비의 관리에 있어서 미세한 의심 증상을 파악해 고장을 예방하는 필요성 터득하였습니다. \n자산설비 엔지니어가 되어 데이터 파악능력을 바탕으로 설비에 축적된 자료를 활용해 설비에 일어날 문제를 예측하여 미리 대응할 것이고 또 새로운 문제에 대한 통찰력을 바탕으로 설비정지로 인한 피해가 없도록 대응할 것입니다."


In [ ]:
# 1) check NULL value
# NULl값이 없더라도 빈 값 유무 확인하기 위해 모든 빈 값을 NULL로 변환하고, 다시 NULL값이 있는지 확인
linkcareer.replace("", float("NaN"), inplace=True)
print(linkcareer.isnull().values.any())

True


In [ ]:
linkcareer.dropna(inplace=True)

In [ ]:
# 길이 확인
print('총 샘플 수 :', len(linkcareer))

총 샘플 수 : 3002


In [ ]:
# 2) 정규표현식 (영어/숫자 기호/문자 제거)
# 텍스트 정제 함수: 한글과 영문 이외의 문자는 모두 제거
def text_cleaning(text):
    hangulenglish = re.compile('[^ \.\(\)\!a-zA-Z\u3131-\u3163\uac00-\ud7a3]+') # 한글과 영문 제외한 모든 글자
    result = hangulenglish.sub('', str(text)) # 해당 글자 공백으로 대체
    return(result)

In [ ]:
cleaned_corpus = []
for sent in linkcareer['content']:
    cleaned_corpus.append(text_cleaning(sent))

linkcareer['content_kor'] = cleaned_corpus
pd.set_option('display.max_colwidth', -1) #truncated되지 않고 post 모두 보여주기
linkcareer.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


year  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
# 3) pykospacing으로 띄어쓰기 교정
from pykospacing import Spacing
spacing = Spacing()

spaced_corpus = []
for sent in linkcareer['content_kor']:
    spaced_corpus.append(spacing(sent))

linkcareer['content_spaced'] = spaced_corpus
linkcareer.head(3)

In [ ]:
# index 재처리 (concat하는 과정에서 index 얽힘)
linkcareer.reset_index(inplace=True)

In [ ]:
# 4) 기업명 불용어 처리
# (주)가 제외된 기업명 stopwords_joo에 저장
stopwords_joo = []
for i in range(len(linkcareer)):
  company_name = linkcareer['company'][i]
  joo = re.compile('[(]+[주]+[)]') # ^[(]+[주]+[)]와  [(]+[주]+[)]$ 모두 포함
  result = joo.sub('', str(company_name)) # 해당 글자 공백으로 대체
  stopwords_joo.append(result)

stopwords_joo

['현대엔지니어링',
 '삼성생명',
 '삼성전자',
 '네이버파이낸셜',
 '수산업협동조합',
 '한국철도공사',
 '충북신용보증재단',
 '현대엠엔소프트',
 'LG에너지솔루션',
 '근로복지공단',
 '국민건강보험공단',
 'Lx세미콘',
 'LG에너지솔루션',
 '보훈병원',
 '씨젠',
 '국민건강보험공단',
 '퍼시스',
 '현대로보틱스',
 '삼성전자',
 'IBK기업은행',
 '삼성전자',
 '롯데정보통신',
 '네이버',
 '웍스모바일',
 '교원그룹',
 'GS칼텍스',
 '동국제강',
 '경기농협',
 '네이버웹툰',
 'KORAIL 한국철도공사',
 '네이버',
 '웍스모바일',
 '교원그룹',
 '포스코케미칼',
 'GS칼텍스',
 '한국자산관리공사',
 '현대모비스',
 'GS리테일',
 '광주은행',
 '서울주택도시공사',
 'LG이노텍',
 '아이비스',
 '근로복지공단',
 '엠케이브이',
 '삼성전자DS',
 '디지지비컬쳐',
 '한국투자증권',
 '서울교통공사',
 '주택도시보증공사',
 '부산교통공사',
 '코레일테크',
 '이베이코리아',
 '지에스티',
 '한국환경공단',
 '현대중공업터보기계',
 '현대글로비스',
 '한국지역난방공사',
 '금융감독원',
 '한국지능정보사회진흥원',
 '한국토지주택공사',
 '라인',
 '기술보증기금',
 '바이오펀치',
 '원익IPS',
 '한국지역난방공사',
 '우리은행',
 '삼성생명',
 'ETAS Korea',
 '한국농어촌공사',
 '한전KPS',
 '한전KPS주식회사',
 '한국서부발전',
 '제주국제자유도시개발센터',
 '포항공대',
 '에스엘',
 '한국장애인고용공단',
 '산업인력공단',
 '한국도로공사',
 '태광산업',
 '주택관리공단',
 '해양수산과학기술진흥원',
 '동원F&B',
 '한국교통안전공단',
 '부천축산농협',
 '메가마트',
 '광주시노인종합사회복지관',
 '한국남부발전',
 '선익시스템',
 '두산퓨얼셀',
 '한국동서발전',
 'KT M모바일'

In [ ]:
# (재)가 제외된 기업명 stopwords_jae에 저장
stopwords_jae = []
for i in range(len(linkcareer)):
  company_name = linkcareer['company'][i]
  joo = re.compile('[(]+[재]+[)]') # ^[(]+[재]+[)]와  [(]+[재]+[)]$ 모두 포함
  result = joo.sub('', str(company_name)) # 해당 글자 공백으로 대체
  stopwords_jae.append(result)

stopwords_jae

['현대엔지니어링',
 '삼성생명',
 '삼성전자',
 '네이버파이낸셜',
 '수산업협동조합',
 '한국철도공사',
 '충북신용보증재단',
 '현대엠엔소프트',
 'LG에너지솔루션',
 '근로복지공단',
 '국민건강보험공단',
 'Lx세미콘',
 'LG에너지솔루션',
 '보훈병원',
 '씨젠',
 '국민건강보험공단',
 '퍼시스',
 '현대로보틱스',
 '삼성전자',
 'IBK기업은행',
 '삼성전자',
 '롯데정보통신',
 '네이버',
 '웍스모바일',
 '교원그룹',
 'GS칼텍스',
 '동국제강',
 '경기농협',
 '네이버웹툰',
 'KORAIL 한국철도공사',
 '네이버',
 '웍스모바일',
 '교원그룹',
 '포스코케미칼',
 'GS칼텍스',
 '한국자산관리공사',
 '현대모비스',
 'GS리테일',
 '광주은행',
 '서울주택도시공사',
 'LG이노텍',
 '아이비스',
 '근로복지공단',
 '엠케이브이(주)',
 '삼성전자DS',
 '디지지비컬쳐',
 '한국투자증권',
 '서울교통공사',
 '주택도시보증공사',
 '부산교통공사',
 '코레일테크(주)',
 '이베이코리아',
 '(주)지에스티',
 '한국환경공단',
 '현대중공업터보기계',
 '현대글로비스',
 '한국지역난방공사',
 '금융감독원',
 '한국지능정보사회진흥원',
 '한국토지주택공사',
 '라인',
 '기술보증기금',
 '바이오펀치',
 '원익IPS',
 '한국지역난방공사',
 '우리은행',
 '삼성생명',
 'ETAS Korea',
 '한국농어촌공사',
 '한전KPS',
 '한전KPS주식회사',
 '한국서부발전',
 '제주국제자유도시개발센터',
 '포항공대',
 '에스엘(주)',
 '한국장애인고용공단',
 '산업인력공단',
 '한국도로공사',
 '태광산업',
 '주택관리공단',
 '해양수산과학기술진흥원',
 '동원F&B',
 '한국교통안전공단',
 '부천축산농협',
 '메가마트',
 '광주시노인종합사회복지관',
 '한국남부발전',
 '선익시스템',
 '두산퓨얼셀',
 '한국동서발전'

In [ ]:
# 그 외 불용어 리스트
word_stopwords = ['글자수','단락','이내','이상','최소','최대','입력가능','공백','포함','작성요령','소제목',
                  '신명조','진하게','글자크기','글자','BYTE','byte','Byte','Bytes']

In [ ]:
# stopwords_joo, stopwords_jae, 기존 기업명 df 합친뒤 중복 제거
stopwords_orig = list(np.array(linkcareer['company'].tolist()))
stopwords = stopwords_joo + stopwords_jae + stopwords_orig + word_stopwords
stopwords_set = set(stopwords) # 집합set으로 변환
stopwords_list = list(stopwords_set) # list로 변환
print(len(stopwords_list)) #총 747개의 기업명 확보

766


In [ ]:
# 불필요한 컬럼 제거
del linkcareer['index']
del linkcareer['content']
# del linkcareer['content_kor']

In [ ]:
# 한 줄에 한 자기소개서 오도록 변경
stopwords_corpus = []
for k in range(len(linkcareer)):
    word_tokens = word_tokenize(linkcareer['content_kor'][k])
    for i in range(len(stopwords_list)):
        for word in word_tokens:
            if stopwords_list[i] in word:
                word_tokens.remove(word)
    stopwords_corpus.append(word_tokens)
    
linkcareer['content_without_stopwords'] = stopwords_corpus

In [ ]:
cleanparen = []
for sent in linkcareer['content_without_stopwords']:
    cleanparen.append(text_cleaning(sent))

linkcareer['content'] = linkcareer['content_without_stopwords'].apply(' '.join)
del linkcareer['content_without_stopwords']

In [ ]:
del linkcareer['year']
del linkcareer['company']
del linkcareer['position']
del linkcareer['content_kor']

def text_cleaning_2(text):
  hangulenglish = re.compile('[^ \.\!a-zA-Z\u3131-\u3163\uac00-\ud7a3]+') # 한글과 영문만
  result = hangulenglish.sub('', str(text))
  return(result)

cleaned_corpus = []
for sent in linkcareer['content']:
    cleaned_corpus.append(text_cleaning_2(sent))

linkcareer['content'] = cleaned_corpus
linkcareer[:3]

content
0  본인이 회사를 선택할때의 기준은 무엇이며 왜 그 기준에 적합한지를 기술해 주십시오 .   건설기술연구원에서 지열원 히트펌프 COP을 올리기 위해 삼 주 정도 기계실에서 상주한 적이 있습니다 . 시설관리자와 함께 설비를 유지 보수하면서 책임감을 느끼고 자신의 설비에 대한 문제를 해결하는 것에 매력을 느꼈고 건물 설비 관리에 관심을 두었습니다.회사란 단순히 돈을 버는 수단이 아닌 회사 성장과 함께 나 자신 자아실천의 장이 되어야 한다고 완성차 철강 공장 등 산업시설과 상업빌딩 의료시설 등 일반건축에 이르기까지 넓은 사업영역을 가지고 있을 뿐만 아니라 해외에서는 개발도상국의 의료시설 산업시설뿐만 아니라 동남아시아 동유럽 등에 수요가 예상되며 해외에 넓은 사업영역을 가지고 있습니다.저는 정적인 자산설비 엔지니어가 되기보단 넓은 영역에서 각각의 설비에 능통하고 최첨단 설비에 적용하여 배우며 폭넓고 높은 지식을 갖고 넓은 분야에서 성과를 올리는 동적인 자산설비 엔지니어가 되고 싶습니다.저는 세 가지 강점이 있습니다첫째 자산설비 엔지니어는 자신이 담당한 설비에 대한 책임감이 있어야 한다고 생각합니다 . 저는 BMTKOREA 현장실습을 할 때 스티어링펌프개조공사중 한 챔버에 달린 두 개의 유압펌프 압력 차로 인해 오일이 누출이 발생하였고 더 큰 피해를 예방하고자 오일을 뒤집어쓰며 손으로 고정해 사장님에게 책임감을 인정받으며 칭찬받은 경험이 있습니다 둘째 설비를 혼자 관리하는 것이 아닌 여러 인원이 관리하기 때문에 원활한 의사소통을 통해 공통적인 목표 지향이 필요합니다 . 저는 인도공과대학교 연구실 인턴을 할 때 자성유체 droplet 프로젝트를 통해 현지 적응 못 하는 팀원을 위해 노력한 적이 있습니다 . 매일 두 번을 같이 써브웨이를 먹었고 매일 저녁에 같이 현지인들과 축구를 하였습니다 . 이를 통해 저는 의사소통을 위해 상대방의 상태에 대한 인식도 중요하다고 배웠습니다 . 셋째 수많은 종류의 설비와 최첨단 설비에 대응하기 위해서는 입사 후에도 배움의 자세가 필요하다고 생각합니다 . 건설기술연구원에서 BEMS를 위한 데이터 수집업무를 주로 하면서 부족함을 많이 느꼈고 실습이 끝나고도 BEMS 협회에서 하는 전문가양성프로젝트에 참여하여 지식을 쌓을 수 있었습니다.이러한 강점을 바탕으로 자산설비 엔지니어가 되어 설비를 유지 보수하고 또한 설비상의 문제를 해결함으로써 고객에게 쾌적한 환경을 제공할 것이고 건물 소유고객의 자산가치를 높이기 위해 노력할 직무분야에 지원하게 된 이유와 선택직무에 본인이 적합하다고 판단할 수 있는 이유 및 근거를 제시해 주십시오   자산설비 기술 엔지니어로서 설비 고장 분석 및 개선을 위해 데이터에 대한 통찰력과 설비 문제 파악 능력이 필요하다고 생각합니다 . 다음과 같은 경험으로 필요한 역량들을 기를 수 있었습니다.분석이 바탕 된 전력소비량 감소학년 여름방학 때 건설기술연구원 건물에너지부서에서 현장실습을 하여 전력소비량 분석을 통해 전체 소비 전력량을 낮춘 경험이 있습니다 . 전력 데이터 시각화 중 년 월부터 구조동 전력량의 기울기 증가를 확인하였습니다 . 년년 월까지 월별로 로우 데이터를 통해 평균 전력량을 비교하였고 시 이후 불필요하게 가동된 항온항습기 때문이란 것을 알았습니다 . 제어 시 전력 이득을 예측하여 관리자에게 중요성을 일깨웠고 올바른 스케줄제어가 되어 결국 달 후 전체 소비 전력량 감소에 일조하였습니다.설비 부품 교체를 통한 COP 향상설비 효율 유지를 위한 BEMS 모니터링 중 기계실 지열원 히트펌프의 효율이 낮아진 것을 발견하였습니다.원인 분석을 위해 관련 논문 편을 통해 유량 입 출구 온도 차 등 주요 파라미터를 확인하여 년도 데이터와 비교 분석하였습니다 . 하지만 오차 범위의 정확한 인지가 어려워 문제점에 대한 방향성을 못 잡았고 이를 매뉴얼을 통해 공급온도 도 환수온도 도에 못 미쳐 펌프 자체에 낮은 효율이 발생하여 생긴 것을 파악했습니다 . 사수와 기계실의 설비를 직접 체크하였고 압축기가 노후화되어 냉매가 누설되고 있었던 것을 확인했습니다 . Leak 이외에도 장비 수명을 고려하여 압축기를 교체하였고 년도 월에 지열 COP을 .까지 올려 효율의 향상을 이루어 냈습니다.이를 통해 데이터 분석 결과 활용의 중요성을 몸소 깨달았고 설비의 관리에 있어서 미세한 의심 증상을 파악해 고장을 예방하는 필요성 터득하였습니다 . 자산설비 엔지니어가 되어 데이터 파악능력을 바탕으로 설비에 축적된 자료를 활용해 설비에 일어날 문제를 예측하여 미리 대응할 것이고 또 새로운 문제에 대한 통찰력을 바탕으로 설비정지로 인한 피해가 없도록 대응할 것입니다 .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
linkcareer.to_csv('/content/drive/MyDrive/21 SUM/KUBIG/NLP 생성 2팀/final 데이터 (전처리 끝)/[기다연] final_preprocessed(자소서 단위, 구두점).csv')

In [ ]:
hangulenglish = re.compile('[^ \.\!a-zA-Z\u3131-\u3163\uac00-\ud7a3]+') # () 제거
stopwords_corpus = hangulenglish.sub('', str(stopwords_corpus))

In [ ]:
# 5-1) 최종 데이터 형태로 변환 (1줄에 1문장이 들어가도록) - 구두점 있는 ver
answer = []
for element in stopwords_corpus:
  answer += element
stopwords_sent = ''.join(answer)
final_list = sent_tokenize(stopwords_sent)

final_data = []
for k in range(len(final_list)):
  final_tokens = re.sub(' \.', '.', final_list[k])
  final_data.append(final_tokens)
final_data[:3]

In [ ]:
# 5-2) 최종 데이터 형태로 변환 (1줄에 1문장이 들어가도록) - 구두점 없는 ver
def shape_differ(text):
  answer = []
  for element in stopwords_corpus:
    answer += element
  stopwords_sent = ' '.join(answer)
  final_list = sent_tokenize(stopwords_sent)
  
  final_data = []
  for i in range(len(final_list)):
    punct = re.compile('[^ \!a-zA-Z\u3131-\u3163\uac00-\ud7a3]+') # 구두점 제거
    final_tokens = punct.sub('', str(final_list[i]))
    final_data.append(final_tokens)
  return(final_data)

fianl_data_nopunc = shape_differ(stopwords_corpus)

In [ ]:
# csv 파일로 변환
import csv

with open('/content/drive/MyDrive/NLP_Generation/linkcareer_preprocessed_final.csv','w') as file :
  write = csv.writer(file)
  write.writerow(final_data)

In [ ]:
with open('preprocessed_no_punc.csv','w') as file :
  write = csv.writer(file)
  write.writerow(fianl_data_nopunc)

### 링커리어 나머지/잡코리아 데이터와 합치기

In [ ]:
# 링커리어 데이터 df로 변경
linkcareer_1 = pd.DataFrame(final_data)
linkcareer_1.columns = (['content'])
linkcareer_1[:3]

,content
0,본인이 회사를 선택할 때의 기준은 무엇이며 왜 그 기준에 적합한지를 기술해 주십시오.
1,건설기술연구원에서 지 열원 히트펌프 COP을 올리기 위해 삼 주 정도 기계실에서 상주 한 적이 있습니다.
2,시설관리자와 함께 설비를 유지 보수하면서 책임감을 느끼고 자신의 설비에 대한 문제를 해결하는 것에 매력을 느꼈고 건물 설비 관리에 관심을 두었습니다.회사란 단순히 돈을 버는 수단이 아닌 회사 성장과 함께 나 자신 자아실천의 장이 되어야 한다고 생각합니다.


In [ ]:
# 링커리어 나머지 데이터
linkcareer_2 = pd.read_csv('/content/drive/MyDrive/21 SUM/KUBIG/NLP 생성 2팀/final 데이터 (전처리 끝)/[전지우](재)preprocessed_final.csv')
del linkcareer_2['Unnamed: 0']
linkcareer_2[:3]

,content
0,지원한 직무와 관련하여 본인의 강점은 무엇이며 본인이 어떻게 기여할 수 있는지를 구...
1,아르바이트로 클린룸에서 Display 설비 운용실을 만들었던 경험이 있습니다.
2,시간 무균 상태를 유지하는 클린룸은 제가 반도체 산업과 CS 직무에 관심을 가지게 ...


In [ ]:
# 잡코리아 데이터
jobkorea = pd.read_csv('/content/drive/MyDrive/NLP_Generation/jobkorea_preprocessed.csv')
del jobkorea['Unnamed: 0']
jobkorea.columns = (['content'])
jobkorea[:3]

,content
0,지원한 가장 큰 이유는 디지털 트랜스포메이션을 적극적으로 진행하여 디지털 혁신을 도 모하는 기업이 기 때문입니다.
1,코로나로 인 한 언택트 시대가 오면서 핀테크를 중심으로 은행의 사업이 더욱 빠르게 변화하고 있습니다.
2,디지털 분야 채용 등으로 이런 흐름에 따라갈 수 있도록 준비하고 있습니다.


In [ ]:
# 데이터 병합
full_data = pd.concat([linkcareer_1, linkcareer_2, jobkorea], axis=0)
full_data.reset_index(inplace=True) # index 재처리
del full_data['index']
full_data

,content
0,본인이 회사를 선택할 때의 기준은 무엇이며 왜 그 기준에 적합한지를 기술해 주십시오.
1,건설기술연구원에서 지 열원 히트펌프 COP을 올리기 위해 삼 주 정도 기계실에서 상주 한 적이 있습니다.
2,시설관리자와 함께 설비를 유지 보수하면서 책임감을 느끼고 자신의 설비에 대한 문제를 해결하는 것에 매력을 느꼈고 건물 설비 관리에 관심을 두었습니다.회사란 단순히 돈을 버는 수단이 아닌 회사 성장과 함께 나 자신 자아실천의 장이 되어야 한다고 생각합니다.
3,완성차 철강 공장 등 산업시설과 상업빌딩 의료시설 등 일반 건축에 이르기까지 넓은 사업 영역을 가지고 있을 뿐만 아니라 해외에서는 개발도상국의 의료시설 산업시설뿐만 아니라 동남아시아 동유럽 등에 수요가 예상되며 해 외에 넓은 사업 영역을 가지고 있습니다.
4,저는 정적인 자산 설비 엔지니어가 되기보단 넓은 영역에서 각각의 설비에 능통하고 최첨단 설비에 적용하여 배우며 폭넓고 높은 지식을 갖고 넓은 분야에서 성과를 올리는 동적인 자산 설비 엔지니어가 되고 싶습니다.
...,...
322219,당시 숫기가 없어 동기들에게 전체적 분위기 주도를 부탁하고 저는 회장으로서 할 수 있는 인원 파악 행사 진행 등에 도 맡았습니다.
322220,이후 더욱 동아리의 친목을 위해 먼저 다가 가는 습관을 들이 고 동아리 문제가 아니어도 개인적인 시간에 함께 어울렸습니다.
322221,현재까지 많은 동아리 원들이 유지되고 활동을 지속해 나가고 있습니 다.
322222,업무에서 이보다 많고 어려운 문제를 겪을 것입니다.


In [ ]:
# 한 줄에 한 문장이 오도록 변환
with open('/content/drive/MyDrive/NLP_Generation/jk_and_link.csv','w') as file :
  write = csv.writer(file)
  write.writerow(full_data)

### 3) GPT-3 모델 fine-tuning
사이트: https://huggingface.co/kykim/gpt3-kor-small_based_on_gpt2
<br>
HuggingFace는 트랜스포머를 기반으로 하는 다양한 모델(transformer.models)과 학습 스크립트(transformer.Trainer)를 구현해 놓은 모듈이다. 원래는 파이토치로 layer, model 등을 선언해주고 학습 스크립트도 전부 구현해야 하지만, 허깅 페이스를 사용하면 이런 수고를 덜 수 있다. 정리하면 '허깅 페이스'라는 회사가 만든 'transformers' 패키지가 있고, 일반적인 파이토치 구현체의 layer.py, model.py이 transformer.models에, train.py 가 transformer.Trainer에 대응된다.

* [BertTokenizerFast](https://huggingface.co/transformers/model_doc/bert.html#berttokenizerfast) <br>
* [BertTokenizerFast : SourceCode](https://huggingface.co/transformers/_modules/transformers/models/bert/tokenization_bert_fast.html#BertTokenizerFast)<br>
* [Kykim/gpt3-kor-small-based_on_gpt2](https://huggingface.co/kykim/gpt3-kor-small_based_on_gpt2)<br>

In [ ]:
# 한 줄에 한 자기소개서 오도록 변환
linkcareer_1 = pd.read_csv('/content/drive/MyDrive/21 SUM/KUBIG/NLP 생성 2팀/final 데이터 (전처리 끝)/[기다연] final_preprocessed(자소서 단위, 구두점).csv')
del linkcareer_1['Unnamed: 0']
linkcareer_1[:2]

,content
0,본인이 회사를 선택할때의 기준은 무엇이며 왜 그 기준에 적합한지를 기술해 주십시오 . 건설기술연구원에서 지열원 히트펌프 COP을 올리기 위해 삼 주 정도 기계실에서 상주한 적이 있습니다 . 시설관리자와 함께 설비를 유지 보수하면서 책임감을 느끼고 자신의 설비에 대한 문제를 해결하는 것에 매력을 느꼈고 건물 설비 관리에 관심을 두었습니다.회사란 단순히 돈을 버는 수단이 아닌 회사 성장과 함께 나 자신 자아실천의 장이 되어야 한다고 완성차 철강 공장 등 산업시설과 상업빌딩 의료시설 등 일반건축에 이르기까지 넓은 사업영역을 가지고 있을 뿐만 아니라 해외에서는 개발도상국의 의료시설 산업시설뿐만 아니라 동남아시아 동유럽 등에 수요가 예상되며 해외에 넓은 사업영역을 가지고 있습니다.저는 정적인 자산설비 엔지니어가 되기보단 넓은 영역에서 각각의 설비에 능통하고 최첨단 설비에 적용하여 배우며 폭넓고 높은 지식을 갖고 넓은 분야에서 성과를 올리는 동적인 자산설비 엔지니어가 되고 싶습니다.저는 세 가지 강점이 있습니다첫째 자산설비 엔지니어는 자신이 담당한 설비에 대한 책임감이 있어야 한다고 생각합니다 . 저는 BMTKOREA 현장실습을 할 때 스티어링펌프개조공사중 한 챔버에 달린 두 개의 유압펌프 압력 차로 인해 오일이 누출이 발생하였고 더 큰 피해를 예방하고자 오일을 뒤집어쓰며 손으로 고정해 사장님에게 책임감을 인정받으며 칭찬받은 경험이 있습니다 둘째 설비를 혼자 관리하는 것이 아닌 여러 인원이 관리하기 때문에 원활한 의사소통을 통해 공통적인 목표 지향이 필요합니다 . 저는 인도공과대학교 연구실 인턴을 할 때 자성유체 droplet 프로젝트를 통해 현지 적응 못 하는 팀원을 위해 노력한 적이 있습니다 . 매일 두 번을 같이 써브웨이를 먹었고 매일 저녁에 같이 현지인들과 축구를 하였습니다 . 이를 통해 저는 의사소통을 위해 상대방의 상태에 대한 인식도 중요하다고 배웠습니다 . 셋째 수많은 종류의 설비와 최첨단 설비에 대응하기 위해서는 입사 후에도 배움의 자세가 필요하다고 생각합니다 . 건설기술연구원에서 BEMS를 위한 데이터 수집업무를 주로 하면서 부족함을 많이 느꼈고 실습이 끝나고도 BEMS 협회에서 하는 전문가양성프로젝트에 참여하여 지식을 쌓을 수 있었습니다.이러한 강점을 바탕으로 자산설비 엔지니어가 되어 설비를 유지 보수하고 또한 설비상의 문제를 해결함으로써 고객에게 쾌적한 환경을 제공할 것이고 건물 소유고객의 자산가치를 높이기 위해 노력할 직무분야에 지원하게 된 이유와 선택직무에 본인이 적합하다고 판단할 수 있는 이유 및 근거를 제시해 주십시오 자산설비 기술 엔지니어로서 설비 고장 분석 및 개선을 위해 데이터에 대한 통찰력과 설비 문제 파악 능력이 필요하다고 생각합니다 . 다음과 같은 경험으로 필요한 역량들을 기를 수 있었습니다.분석이 바탕 된 전력소비량 감소학년 여름방학 때 건설기술연구원 건물에너지부서에서 현장실습을 하여 전력소비량 분석을 통해 전체 소비 전력량을 낮춘 경험이 있습니다 . 전력 데이터 시각화 중 년 월부터 구조동 전력량의 기울기 증가를 확인하였습니다 . 년년 월까지 월별로 로우 데이터를 통해 평균 전력량을 비교하였고 시 이후 불필요하게 가동된 항온항습기 때문이란 것을 알았습니다 . 제어 시 전력 이득을 예측하여 관리자에게 중요성을 일깨웠고 올바른 스케줄제어가 되어 결국 달 후 전체 소비 전력량 감소에 일조하였습니다.설비 부품 교체를 통한 COP 향상설비 효율 유지를 위한 BEMS 모니터링 중 기계실 지열원 히트펌프의 효율이 낮아진 것을 발견하였습니다.원인 분석을 위해 관련 논문 편을 통해 유량 입 출구 온도 차 등 주요 파라미터를 확인하여 년도 데이터와 비교 분석하였습니다 . 하지만 오차 범위의 정확한 인지가 어려워 문제점에 대한 방향성을 못 잡았고 이를 매뉴얼을 통해 공급온도 도 환수온도 도에 못 미쳐 펌프 자체에 낮은 효율이 발생하여 생긴 것을 파악했습니다 . 사수와 기계실의 설비를 직접 체크하였고 압축기가 노후화되어 냉매가 누설되고 있었던 것을 확인했습니다 . Leak 이외에도 장비 수명을 고려하여 압축기를 교체하였고 년도 월에 지열 COP을 .까지 올려 효율의 향상을 이루어 냈습니다.이를 통해 데이터 분석 결과 활용의 중요성을 몸소 깨달았고 설비의 관리에 있어서 미세한 의심 증상을 파악해 고장을 예방하는 필요성 터득하였습니다 . 자산설비 엔지니어가 되어 데이터 파악능력을 바탕으로 설비에 축적된 자료를 활용해 설비에 일어날 문제를 예측하여 미리 대응할 것이고 또 새로운 문제에 대한 통찰력을 바탕으로 설비정지로 인한 피해가 없도록 대응할 것입니다 .
1,금융 백종원 백종원은 여러 매체를 통해 요리에 관한 각종 정보와 비결을 공유하며 요 리의 트렌드를 이끌고 있습니다 . 금융은 요리보다 더 정보의 비대칭으로 말미암은 손실이 큰 분야입니다 . 저는 평소에 비전공자인 주위 사람들에 게 다양한 금융상품의 리스크와 리턴을 알기 쉽게 전달하여 금융 백종원 이라는 별명을 가지고 있습니다 . 이 경험을 바탕으로 일원이 되어서도 일상적인 업무 외에 적극 사회공헌활동에 참여하고자 하며 금 융 취약계층이 정보의 비대칭으로 입는 손실을 줄여주는 든든한 이웃으로 자리 잡아 사회적 가치를 더욱 높일 수 있는 구성원이 되겠습 니다 . 유쾌한 소통을 통한 친화력은 제 큰 장점입니다 . 성인이 된 이후로 방학 휴학기간을 이용하여 국내외의 다양한 서비스직을 경험했습니다.또한 근 로 장학생으로 근무할 당시 직원과 학생들의 소통을 주도 하였습니다 . 사 실을 기반으로 서로의 상황을 이해 정리함으로써 중간에서 이견을 조율 하고 조직의 단결을 이끌어내는 역할을 수행해 본 경험으로 협업에 자신 있습니다.사람을 잘 믿고 거절을 못 하는 습관이 업무현장에서 단점으로 작용한 적 이 있었습니다 . 허나 두 차례의 금융기관 인턴을 진행하며 업무에서만큼 은 맹목적인 신뢰가 단점이 될 수 있음을 알게 되었습니다 . 이후 동료나 고객의 말을 정리해서 되물어 일차적으로 재확인하는 절차가 습관이 되었 고 내규와 법령을 직접 확인한 후 실행하려 노력해 보완할 수 있었습니 다 . 기업금융 전문가 회사와 나 나와 고객이 함께 성장하는 상생의 가치를 믿습니다 . 은 최고의 브랜드 파워로 남다른 고객기반을 보유하고 있습니다 . 그 때문 에 많은 현장경험으로 나를 한층 더 발전시킬 수 있을 것이라 생각해 지원 을 결심했습니다 . 저는 경제학을 전공하고 회계분야의 꾸준한 노력으로 경제주체인 개인 기업 정부에 대한 충분한 경제적 이해를 선행했습니다 . 나아가 기업은행과 인턴경험을 쌓으며 기업여신 신용 보증보험 등 기업금융에 대한 다양한 실무지식을 쌓았습니다 . 기업규제 법과 코로나상황으로 기업의 배상리스크가 증가했으며 종업원과 경영진 의 건강에 대한 관심이 커진 상황에서 기업고객에게 보험 대출 신탁등 충분한 정보를 제공하여 재무관리를 돕는 한편 기업고객들의 니즈를 파 악하여 그들을 위한 혁신금융모델을 구축하는데 기여하는 사원으로 성장 하고 싶습니다 .


In [ ]:
# 링커리어 (전지우)
linkcareer_2 = pd.read_csv('/content/drive/MyDrive/21 SUM/KUBIG/NLP 생성 2팀/final 데이터 (전처리 끝)/[전지우] final_preprocessed(자소서 단위).csv')
del linkcareer_2['Unnamed: 0']
linkcareer_2[:2]

,content
0,지원한 직무와 관련하여 본인의 강점은 무엇이며 본인이 어떻게 기여할 수 있는 지를 구체적으로 기술하여 주십시오.아르바이트로 클린룸에서 Display 설비 운용실을 만들었던 경험이 있습니다 . 간 무균 상태를 유지하는 클린룸은 제가 반도체 산업과 CS 직무에 관심을 가지게 된 계기가 되었습니다 . CS 엔지니어는 반도체 전공에 대한 폭넓은 이해와 트러블 슈팅에 상시 대응할 수 있는 체력이 필요하다 생각합니다 . 그래서 저는 직무에 적합한 두 가지 역량을 쌓기 위해 다음과 같은 노력을 했습니다.첫째 에너지화학 및 공학 수업으로 PNjunction MOSFET과 같은 반도체 소자에 관한 기본 지식을 쌓고 스스로 온 강의를 찾아 들으며 공정 관련 지식을 쌓았습니다 . 하지만 이것만으로는 부족하다 생각해 현재 NCS 주관 반도체 공정 강의를 추가로 학습하고 있습니다.둘째 학창시절 육상 선수 생활을 하며 기초 체력을 만들었습니다 . 선수 생활이 끝나고도 대학 간 꾸준히 축구 동아리 활동을 하며 체력을 유지했고 군인 때 측정한 m 달리기에서 라는 기록으로 대대에서 을 할 수 있었습니다.저는 이러한 저의 전공 지식과 강한 체력으로 긴급 oncall에 항시 대응할 수 있도록 준비하고 발생한 문제를 빠르게 해결하는 CS 엔지니어가 되도록 노력하겠습니다 . 인재상 중 본인을 가장 잘 표현하는 한 단어를 선택하고 그 이유에 대해 경험 위주로 서술해 주십시오.인턴의 신선한 인재상 중 저를 가장 잘 표현하는 단어는 Leading Innovation이라고 생각합니다 . 간 인턴을 하며 새로운 유전체 파우더의 양산화 과정을 진행했습니다 . 기존의 파우더는 MLCC의 수요 증가로 원하는 공급량을 맞출 수 없어 새로운 파우더로 동등 특성을 구현해야 했습니다 . 하지만 새로운 파우더는 기존 양산 조건에서 과하게 소결되어 더 높은 DF와 DCBAIS 저하를 발생시켜 MLCC의 성능 및 신뢰성 저하를 만들었습니다 . 원하는 전기적 특성을 구현하기 위해 기존의 양산품 조건과는 다른 소성 온도 해쇄 강도 첨가제 등 다양한 조건들을 적절히 조절하였습니다 . Sampling 한 MLCC는 Polishing 후 소결 정도를 SEM으로 직접 확인하고 Flash 물 끓임과 같은 신뢰성 테스트를 진행해 원하는 특성을 찾았습니다 . 그리고 인턴 마지막 주엔 그동안의 업무 진행을 발표하면서 파우더의 입자 균일성을 유지하기 위해 슬러리를 원심분리기에 넣어 기준치보다 작고 가벼운 파우더 입자를 분리하는 방법을 발표했습니다 . 비록 채택되지는 못하였지만 제가 맡은 업무에 대해 끊임없이 고민하고 문제의 원인과 결과를 분석하여 스스로 계속해서 발전하고자 노력하였습니다 . 살면서 가장 소속감을 느낀 조직은 무엇이었으며 그 조직의 발전을 위해 헌신적으로 노력했던 것 중 가장 기억에 남는 경험에 대해 기술해 주십시오 . 쪽방에서 우산 비닐 줄이기 프로젝트청년 창업에 대한 지원 강화와 교내 지원 프로그램으로 창업에 관심을 가져 년 기부터 창업동아리 활동을 진행하며 안에 하나의 특허 등록하기라는 목표를 세웠습니다 . 친환경을 키워드로 하는 이번 정부의 국정 과제에 초점을 맞춰 국내에서 매년 장 소비되는 우산 비닐 사용을 줄이기 위해 우산 탈수기를 고안했습니다.원심분리기의 원리와 원통형 대걸레 탈수기의 페달 작동법을 사용하여 물기를 제거하는 이 우산 전용 탈수기는 물기는 제거할 수 있지만 원심력이 강해지면서 얇은 우산살을 모두 부러뜨려 우산으로써 제 기능을 할 수 없게 만들었습니다 . 그래서 저희는 우산살이 부러지는 것을 막기 위해 우산의 부피에 맞춰 둘레가 조절 가능한 벽을 설치하여 회전 시 우산살로 가는 힘을 분산시킬 수 있었습니다.단점을 보완하고 프린터로 프로토타입을 만들어 특허 등록을 진행하였지만 이미 특허청에 등록된 아이디어였기 때문에 아쉽게도 원하는 결과를 얻을 수 없었습니다.하지만 주인의식을 바탕으로 함께 운영해왔던 동아리였기 때문에 구성원 모두 자발적으로 늦은 시간까지 회의에 참여하고 학업 중에도 따로 시간을 만들어 창업 박람회 및 교내 창업 경진대회에 참가했습니다 . 이 경험을 통해 적절한 동기부여가 얼마나 중요한지와 어떻게 하면 팀원들과 원활히 소통할 수 있는지 배울 수 있었던 값진 기회였습니다 . 지금까지 살아오면서 타인과 차별화되는 경험 또는 경력이 있다면 어떤 것인지 구체적으로 서술해 주십시오.간의 세계 배낭여행버킷리스트인 배낭여행에 다녀오기 위해 학기 중 하나의 아르바이트에서 근무하며 원의 경비를 마련해 작년 인도 포르투갈 모로코 등 국으로 간의 배낭여행을 다녀올 수 있었습니다.저는 이 여행을 통해 두 가지 배움을 가지고 돌아올 수 있었습니다.첫째 어떠한 상황에서도 적응하는 법을 배웠습니다 . 배낭여행 중에는 열악한 환경에 제대로 된 숙박 시설과 대중교통이 갖춰져 있지 않은 곳들이 많았습니다 . 또한 부족한 경비 때문에 처음 만나는 여행객들과 자연스럽게 숙소를 나눠 쓰고 세상에서 가장 비위생적인 곳이라 생각했던 인도에서 매 끼니를 현지 식당에서 해결하며 어렵지 않게 적응할 수 있었습니다.둘째 돌발상황에 유연하게 대처하는 법을 배웠습니다 . 돌아가는 차편을 놓쳤던 날 히치하이킹으로 숙소로 돌아왔었고 갑작스러운 강풍으로 배가 출항하지 않았던 날 기상의 영향을 덜 받는 큰 규모의 배가 있는 항구를 찾아 이동했습니다.이러한 경험들은 훗날 업무 중 문제 상황이 발생했을 때 당황하지 않고 체계적으로 해결해 나갈 수 있는 원동력이 될 것입니다 .
1,직무활동 동아리동호회 팀프로젝트 연구회 재능기부 등 주요 직무능력을 서술하여 주십시오 . 마음을 열다 대학교 년 때 마케팅 수업에서 해외여행 상품 개발 관련 프로젝트를 진행하면서 조장을 맡았습니다 . 타 학과 및 나이가 서로 다른 팀원들로 구성되어 짧은 시간 안에 끝내야 하는 프로젝트를 하는 데 어려움이 있을 것으로 생각하였습니다 . 하지만 조급해하지 않으며 서로를 알아가는 시간을 가지고 팀원 개개인의 효율을 극대화할 수 있는 역할을 분담하여 진행하였습니다 . 또한 잦은 미팅을 통해 소통하여 개인의 역할 뿐만 아니라 모두가 이해할 수 있게 도왔습니다 . 팀 프로젝트 과정에서 항상 갈등은 발생하지만 이 역시 소통하지 않으면 갈등은 더 풀 수 없는 실타래가 된다고 생각합니다 . 그래서 저는 소통과 대화가 절실히 필요하다고 생각하였고 여러 관점에서 의견을 수용하는 분위기를 만들었습니다 . 그로 인해 프로젝트에서 좋은 결과를 얻을 수 있었습니다 . 한 해의 시작을 꽃과 함께 손재주가 좋고 새로운 것에 관심을 가지는 성격으로 스스로 만든 제품을 판매한 적이 있습니다 . 그 자체만으로 사람들을 행복하게 해주는 꽃이라는 소재로 달력을 만들어보고 싶다는 생각을 했고 빠른 실행력과 추진력으로 혼자 발주부터 제작 홍보 배송까지 책임감을 가지고 한 결과 두 달 동안 재료값의 이익을 얻을 수 있었습니다 . 도전을 어려워하지 않고 즐겼더니 스스로에 대한 확신이 강해지고 어떤 일이든 잘해낼 수 있다는 자신감이 생겼습니다 . 공동체의 일원으로서 가장 중요하게 생각하는 가치를 제시하고 그 가치를 실천했던 경험을 기술하여 주십시오 . 공동체 안에서 가장 중요한 것은 약속의 이행이라고 생각합니다 . 약속 중에서 가장 중요하게 지켜야 할 것은 시간약속입니다 . 신뢰나 믿음의 바탕은 시간약속에서부터 시작한다고 생각합니다 . 약속 시각 전에 나와 있는 것은 쉬워 보여도 사실 어려운 일이기도 합니다 . 저는 아르바이트를 할 때 당연하지만 늘 전에 도착해서 근무교대를 하거나 일 할 자세를 갖추었습니다 . 아르바이트했을 때 지켰던 시간 엄수가 친구들과의 약속에도 항상 적용되었고 제가 제시간에 도착하니 자주 늦었던 친구들도 시간 약속을 잘 지키게 되었습니다 . 공동체 생활을 할 때는 남에게 절대 피해를 주면 안 된다 생각하고 그중 가장 큰 부분을 차지하는 게 시간약속이라고 생각합니다 . 처음 만났거나 잘 알지 못하는 상대를 위해 본인이 먼저 배려했던 경험 한 가지를 언급하고 상대방을 위해 그런 행동을 하게 된 이유를 함께 기술하여 주십시오 . 티 나지 않는 마음 눈이 펑펑 내리는 한겨울 고등학생이었던 저는 야간자율학습을 마치고 집에 가고 있었습니다 . 아파트단지에

In [ ]:
# 잡코리아 (이현규)
jobkorea = pd.read_csv('/content/drive/MyDrive/21 SUM/KUBIG/NLP 생성 2팀/final 데이터 (전처리 끝)/[이현규] final_preprocessed(자소서 단위)_수정.csv')
del jobkorea['Unnamed: 0']
jobkorea[:2]

,content
0,지원한 가장 큰 이유는 디지털 트랜스포메이션을 적극적으로 진행하여 디지털 혁신을 도모하는 기업이기 때문입니다 . 코로나로 인한 언택트 시대가 오면서 핀테크를 중심으로 은행의 사업이 더욱 빠르게 변화하고 있습니다 . 디지털 분야 채용 등으로 이런 흐름에 따라갈 수 있도록 준비하고 있습니다 . 특히 금융산업 빅데이터는 데이터 보유량이 많고 그 증가 속도가 빠릅니다 . 이는 곧 다양한 분석 기회의 제공으로 저의 분석 역량을 키우고 같이 성장할 기회가 될 것입니다 . 디지털 그룹으로 배치 후 영업현장으로 파견 가는 것 역시 선택한 이유 중 하나입니다 . 대부분 은행은 바로 영업현장으로 파견되어 분석 역량을 유지하기 위해 무엇을 해야 할지 고민이 될 것 같았습니다 . 그러나 배치된 곳에 맞춰 미리 관련 역량을 준비할 수 있을 것입니다 . 입행 후 추천시스템 개발이나 자연어 처리의 전문가가 되고 싶습니다 . 금융상품의 추천시스템 개발로 고객이 바로 체감할 수 있는 서비스를 개발하고 싶습니다 . 혹은 자연어 처리를 통해 사용자의 거래 내역을 분류하거나 AI 상담 서비스 향상에 이바지하고 싶습니다 . 호기심 제가 가지고 있는 디지털 DNA는 호기심입니다 . 호기심은 어떤 분석 문제를 찾아내는 원동력입니다 . 이런 호기심으로 년 중등교사 일반사회 임용시험의 데이터를 분석한 적이 있습니다 . 친구가 임용시험의 지원 지역으로 OOO를 고민했습니다 . OOO의 모집인원이 많아 커트라인이 낮아질 것으로 생각했기 때문입니다 . 저는 이러한 예상을 데이터로 확인해보고 싶었습니다 . 총 개의 지역을 분석했고 각각 년의 데이터를 활용하였습니다 . 변수들이 많지 않아 각 변수의 관계를 파악하기 위해 산점도를 그렸습니다 . 산점도를 보았을 때 지역별로 지원자 수와 커트라인 간에 패턴이 있다고 판단되어 지역별로 분석했습니다 . 물론 지역별 표본이 많지 않고 모든 지역을 분석한 것이 아니라는 한계가 있었습니다 . 그러나 호기심으로 찾아낸 시험 커트라인 예상이라는 문제를 분석으로 해결해본 좋은 경험이 되었습니다 . 데이터 문제로 전환하여 해결하는 역량 호기심이라는 디지털 DNA로 분석 문제를 찾아내도 데이터 문제로 전환하여 해결하는 역량이 없다면 의미가 없습니다 . OO시 공공자전거 추가 대여소 위치 제안이라는 주제로 분석 프로젝트를 진행하여 이러한 역량을 키웠습니다 . 프로젝트의 관건은 어떻게 추가 대여소의 위치 제안을 데이터 문제로 해결할 수 있을까였습니다 . 그래서 생각해낸 것이 시설물의 위치와 기상 데이터로 특정 지점의 월별 대여량을 예측하는 모델을 만드는 것이었습니다 . 선형 회귀 분석의 전진 선택법으로 총 개의 변수를 선택하고 예측 모델을 만들었습니다 . 이를 인구밀도 기준으로 선정한 후보군에 적용하여 대여량을 예측했습니다 . 추가로 버스 운영 대수로 교통 소외지역을 시각화해 임의로 곳의 대여소를 더 제안했습니다 . 위의 과정에서 제가 수행한 역할은 주로 데이터 수집과 전처리였습니다 . 특히 전처리에서 시설물의 위치 정보를 특정 지점 근처에 존재하는 개수 등으로 변환하는 작업이 뿌듯했습니다 . 또한 예측의 효율을 위해 선정된 후보군을 Kmeans Clustering으로 축소하는 작업을 역시 흥미로웠습니다 . 일하면서 저의 디지털 DNA인 호기심으로 분석 문제를 찾겠습니다 . 그리고 그 문제를 데이터 문제로 전환하여 해결하는 데 이바지하겠습니다 . 입출금리포트 서비스 개선 사용 내역 분류 우리WON뱅킹의 입출금리포트 서비스를 개선해보고 싶습니다 . 그중에서도 주요지출 분석에서 고객의 카드대금 사용 내역을 더욱 분류하여 제공하고 싶습니다 . 예를 들어 한 달에 식비가 얼마나 되는지 교통비로 얼마를 쓰는지 등을 분류하여 제공합니다 . 특히 요즘 구독 경제의 활성화로 구독 서비스에 적지 않은 금액의 소비를 하고 있습니다 . 따라서 구독 서비스에 소비하는 금액을 분류하여 제시한다면 소비 측면에서의 고객 자산 관리를 효과적으로 도울 수 있을 것입니다 . 이러한 서비스를 제공하기 위해서는 사용 내역이 정확하게 분류되어있는 데이터가 필요합니다 . 이 훈련 데이터를 얻기 위해서 원터치 알림 앱을 활용할 수 있을 것입니다 . 지금처럼 간단한 메모 기능을 제공하는 것이 아니라 미리 정해놓은 카테고리를 사용자가 라벨링할 수 있는 기능을 제공합니다 . 그리고 우선 원터치 알림 앱에서 대시보드를 제공하여 고객이 적극적으로 사용 내역을 분류할 수 있도록 해야 할 것입니다 . 이렇게 해당 서비스를 충분한 데이터가 쌓일 때까지 지속한다면 질 좋은 훈련 데이터를 얻을 수 있을 것입니다 . 마지막으로 획득한 훈련 데이터를 자연어 처리를 통해 분류 모델을 만들면 고객의 카드대금 사용 내역 분류 서비스를 제공할 수 있을 것입니다 .
1,직장인에게 가장 필요한 부분은 소통이라고 생각합니다 . 직장에서 발생하는 문제점 중 소통의 부재가 원인인 경우가 많아지고 있습니다 . 소통의 부재로 발생하는 구성원간의 갈등은 조직의 사기 저하와 업무생산성에 부정적인 영향을 미치게 될 것입니다 . 저는 대학교 동아리 내에서 팀원 소통의 시간을 준비했던 경험이 있습니다 . 유익한 시간을 만들기 위해 팀원별 원하는 활동을 조사하였고 최종적으로 오페라와 토크 콘서트 관람이 선정되었습니다 . 차 활동 후 티타임을 가지면서 어려웠던 부분을 공유하고 적절한 대처방안을 모색하였습니다 . 그리고 의견 충돌이 있었던 경우는 대화를 통해 배려하고 상대방 입장에서 생각하게 되는 시간을 가지게 되었습니다 . 이러한 소통의 모습을 통해 화합하는 분위기를 만들 수 있었습니다 . 농협 직원이 된다면 고객 및 동료들과 소통 문화를 정착시켜 최상의 행복 가치를 위해 최선을 다하는 행복 파트너가 되고자 합니다 . 어떤 일을 처리함에 있어서 시간분배를 최우선으로 생각합니다 . 간혹 회사 내에서 긴급으로 요구되는 업무나 예상치 못한 개인사와 같은 요인들이 방해요소로 작용하곤 합니다 . 평상시 중요도를 파악한 후 적절한 시간분배를 통해 빠르게 처리할 수 있는 부분부터 계획을 짜는 습관을 들여 어떤 방해요소가 있더라도 차질이 생기지 않도록 노력하고 있습니다 . 제가 대학시절 학교시험 및 자격증 취득을 위해 공부를 할 때도 적정한 시간분배 계획을 활용하였습니다 . 요일별로 공부할 과목을 분류한 후 비교적 쉬운 과목과 어려운 과목의 비중을 이나 의 비율로 나누어 진행하였습니다 . 효율적인 시간분배를 통해 쉬운 부분은 빠르게 처리하면서 부족한 부분에 대해서 더 많은 시간을 투자할 수 있었습니다 . 이러한 노력으로 과내에서 상위권 성적을 유지할 수 있었으며 장학금 수혜 및 여러 가지 교내지원 프로그램에 참여하는 기회도 부여 받았습니다 . 만약 농협에 입사하게 된다면 회사 내외부 업무를 담당하여도 제가 중요시 하는 시간분배 계획을 통해 문제없이 처리할 수 있다고 자신합니다 . 동아리 내 함께 운동을 즐겨하던 친구들과 함께 농구 대회에 참가한 적이 있었습니다 . 긴 시간 발을 맞춰온 터라 예선전을 순조롭게 통과하여 상위 토너먼트에 진출하였습니다 . 하지만 하루에 많은 경기를 치르다보니 팀원들의 체력이 점차 소진되기 시작했습니다 . 더군다나 상대편의 거칠면서 비신사적인 행동은 저희들을 육체적 정신적으로 힘들게 만들었습니다 . 그런 영향인지 저와 팀원들은 잦은 실수를 하며 경기를 어렵게 치렀습니다 . 당시 주장이었던 저는 팀원들을 모아 상대편의 어떤 행동에도 흔들리지 말고 우리만의 플레이를 하면서 기회를 만들어가자고 긍정적인 마인드를 심어주었습니다 . 경기 중간에 팀원들이 평정심을 유지하도록 응원을 하였고 그런 노력의 결과가 조금씩 나타났습니다 . 점차 팀플레이가 안정적으로 잡혀갔으며 경기를 승리할 수 있었습니다 . 더 높은 자리까지 올라가진 못하였으나 힘든 상황에도 승리를 해준 친구들에게 고마움을 느꼈습니다 . 어려운 환경에서도 긍정적인 마음은 성취의 원동력이 된다고 생각합니다 . 긍정적인 생각과 열린 마음으로 구성원간의 협력을 통해 성과가 극대화될 수 있도록 노력하는 인재가 되겠습니다 . 사회가 변화하면서 소비자들의 구매형태가 다양해졌습니다 . 소비자는 구매하고자 하는 농산물 및 물품에 대하여 여러 가지 방법으로 정보를 수집한 후 구매를

In [ ]:
# 데이터 병합
full_data = pd.concat([linkcareer_1, linkcareer_2, jobkorea], axis=0)
full_data.reset_index(inplace=True) # index 재처리
del full_data['index']
full_data

content
0      본인이 회사를 선택할때의 기준은 무엇이며 왜 그 기준에 적합한지를 기술해 주십시오 .   건설기술연구원에서 지열원 히트펌프 COP을 올리기 위해 삼 주 정도 기계실에서 상주한 적이 있습니다 . 시설관리자와 함께 설비를 유지 보수하면서 책임감을 느끼고 자신의 설비에 대한 문제를 해결하는 것에 매력을 느꼈고 건물 설비 관리에 관심을 두었습니다.회사란 단순히 돈을 버는 수단이 아닌 회사 성장과 함께 나 자신 자아실천의 장이 되어야 한다고 완성차 철강 공장 등 산업시설과 상업빌딩 의료시설 등 일반건축에 이르기까지 넓은 사업영역을 가지고 있을 뿐만 아니라 해외에서는 개발도상국의 의료시설 산업시설뿐만 아니라 동남아시아 동유럽 등에 수요가 예상되며 해외에 넓은 사업영역을 가지고 있습니다.저는 정적인 자산설비 엔지니어가 되기보단 넓은 영역에서 각각의 설비에 능통하고 최첨단 설비에 적용하여 배우며 폭넓고 높은 지식을 갖고 넓은 분야에서 성과를 올리는 동적인 자산설비 엔지니어가 되고 싶습니다.저는 세 가지 강점이 있습니다첫째 자산설비 엔지니어는 자신이 담당한 설비에 대한 책임감이 있어야 한다고 생각합니다 . 저는 BMTKOREA 현장실습을 할 때 스티어링펌프개조공사중 한 챔버에 달린 두 개의 유압펌프 압력 차로 인해 오일이 누출이 발생하였고 더 큰 피해를 예방하고자 오일을 뒤집어쓰며 손으로 고정해 사장님에게 책임감을 인정받으며 칭찬받은 경험이 있습니다 둘째 설비를 혼자 관리하는 것이 아닌 여러 인원이 관리하기 때문에 원활한 의사소통을 통해 공통적인 목표 지향이 필요합니다 . 저는 인도공과대학교 연구실 인턴을 할 때 자성유체 droplet 프로젝트를 통해 현지 적응 못 하는 팀원을 위해 노력한 적이 있습니다 . 매일 두 번을 같이 써브웨이를 먹었고 매일 저녁에 같이 현지인들과 축구를 하였습니다 . 이를 통해 저는 의사소통을 위해 상대방의 상태에 대한 인식도 중요하다고 배웠습니다 . 셋째 수많은 종류의 설비와 최첨단 설비에 대응하기 위해서는 입사 후에도 배움의 자세가 필요하다고 생각합니다 . 건설기술연구원에서 BEMS를 위한 데이터 수집업무를 주로 하면서 부족함을 많이 느꼈고 실습이 끝나고도 BEMS 협회에서 하는 전문가양성프로젝트에 참여하여 지식을 쌓을 수 있었습니다.이러한 강점을 바탕으로 자산설비 엔지니어가 되어 설비를 유지 보수하고 또한 설비상의 문제를 해결함으로써 고객에게 쾌적한 환경을 제공할 것이고 건물 소유고객의 자산가치를 높이기 위해 노력할 직무분야에 지원하게 된 이유와 선택직무에 본인이 적합하다고 판단할 수 있는 이유 및 근거를 제시해 주십시오   자산설비 기술 엔지니어로서 설비 고장 분석 및 개선을 위해 데이터에 대한 통찰력과 설비 문제 파악 능력이 필요하다고 생각합니다 . 다음과 같은 경험으로 필요한 역량들을 기를 수 있었습니다.분석이 바탕 된 전력소비량 감소학년 여름방학 때 건설기술연구원 건물에너지부서에서 현장실습을 하여 전력소비량 분석을 통해 전체 소비 전력량을 낮춘 경험이 있습니다 . 전력 데이터 시각화 중 년 월부터 구조동 전력량의 기울기 증가를 확인하였습니다 . 년년 월까지 월별로 로우 데이터를 통해 평균 전력량을 비교하였고 시 이후 불필요하게 가동된 항온항습기 때문이란 것을 알았습니다 . 제어 시 전력 이득을 예측하여 관리자에게 중요성을 일깨웠고 올바른 스케줄제어가 되어 결국 달 후 전체 소비 전력량 감소에 일조하였습니다.설비 부품 교체를 통한 COP 향상설비 효율 유지를 위한 BEMS 모니터링 중 기계실 지열원 히트펌프의 효율이 낮아진 것을 발견하였습니다.원인 분석을 위해 관련 논문 편을 통해 유량 입 출구 온도 차 등 주요 파라미터를 확인하여 년도 데이터와 비교 분석하였습니다 . 하지만 오차 범위의 정확한 인지가 어려워 문제점에 대한 방향성을 못 잡았고 이를 매뉴얼을 통해 공급온도 도 환수온도 도에 못 미쳐 펌프 자체에 낮은 효율이 발생하여 생긴 것을 파악했습니다 . 사수와 기계실의 설비를 직접 체크하였고 압축기가 노후화되어 냉매가 누설되고 있었던 것을 확인했습니다 . Leak 이외에도 장비 수명을 고려하여 압축기를 교체하였고 년도 월에 지열 COP을 .까지 올려 효율의 향상을 이루어 냈습니다.이를 통해 데이터 분석 결과 활용의 중요성을 몸소 깨달았고 설비의 관리에 있어서 미세한 의심 증상을 파악해 고장을 예방하는 필요성 터득하였습니다 . 자산설비 엔지니어가 되어 데이터 파악능력을 바탕으로 설비에 축적된 자료를 활용해 설비에 일어날 문제를 예측하여 미리 대응할 것이고 또 새로운 문제에 대한 통찰력을 바탕으로 설비정지로 인한 피해가 없도록 대응할 것입니다 .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 2.8 MB 10.9 MB/s 
     |████████████████████████████████| 895 kB 40.5 MB/s 
     |████████████████████████████████| 636 kB 40.6 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 3.3 MB 39.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Tokenizer 설정
from transformers import BertTokenizerFast, GPT2LMHeadModel

# BERT tokenizer 설정
# tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
tokenizer_gpt3 = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2", 
                                                   bos_token='<|startoftext|>', 
                                                   eos_token='<|endoftext|>', 
                                                   pad_token='<|pad|>')

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print("본 모델의 최대 길이는 {} 입니다.".format(tokenizer_gpt3.model_max_length))
print("Sequence token {}의 처음은 {} id를 갖고 있습니다.".format(tokenizer_gpt3.convert_ids_to_tokens(tokenizer_gpt3.bos_token_id), tokenizer_gpt3.bos_token_id))
print("Sequence token {}의 끝은 {} id를 갖고 있습니다.".format(tokenizer_gpt3.convert_ids_to_tokens(tokenizer_gpt3.eos_token_id), tokenizer_gpt3.eos_token_id))
print("Padding token {}은 {} id를 갖고 있습니다.".format(tokenizer_gpt3.convert_ids_to_tokens(tokenizer_gpt3.pad_token_id), tokenizer_gpt3.pad_token_id))

본 모델의 최대 길이는 2048 입니다.
Sequence token <|startoftext|>의 처음은 42000 id를 갖고 있습니다.
Sequence token <|endoftext|>의 끝은 42001 id를 갖고 있습니다.
Padding token <|pad|>은 42002 id를 갖고 있습니다.


In [ ]:
# 예시 문장
example = full_data['content'][0]
print(example)

print(tokenizer_gpt3.encode('<|startoftext|>' + example + '<|endoftext|>', 
      truncation=True, max_length=768, padding="max_length")[1:])
# truncation: 최대 길이보다 문장이 길면 자름
# max_length: 최대 길이만큼 길이 재조정

본인이 회사를 선택할때의 기준은 무엇이며 왜 그 기준에 적합한지를 기술해 주십시오 .   건설기술연구원에서 지열원 히트펌프 COP을 올리기 위해 삼 주 정도 기계실에서 상주한 적이 있습니다 . 시설관리자와 함께 설비를 유지 보수하면서 책임감을 느끼고 자신의 설비에 대한 문제를 해결하는 것에 매력을 느꼈고 건물 설비 관리에 관심을 두었습니다.회사란 단순히 돈을 버는 수단이 아닌 회사 성장과 함께 나 자신 자아실천의 장이 되어야 한다고 완성차 철강 공장 등 산업시설과 상업빌딩 의료시설 등 일반건축에 이르기까지 넓은 사업영역을 가지고 있을 뿐만 아니라 해외에서는 개발도상국의 의료시설 산업시설뿐만 아니라 동남아시아 동유럽 등에 수요가 예상되며 해외에 넓은 사업영역을 가지고 있습니다.저는 정적인 자산설비 엔지니어가 되기보단 넓은 영역에서 각각의 설비에 능통하고 최첨단 설비에 적용하여 배우며 폭넓고 높은 지식을 갖고 넓은 분야에서 성과를 올리는 동적인 자산설비 엔지니어가 되고 싶습니다.저는 세 가지 강점이 있습니다첫째 자산설비 엔지니어는 자신이 담당한 설비에 대한 책임감이 있어야 한다고 생각합니다 . 저는 BMTKOREA 현장실습을 할 때 스티어링펌프개조공사중 한 챔버에 달린 두 개의 유압펌프 압력 차로 인해 오일이 누출이 발생하였고 더 큰 피해를 예방하고자 오일을 뒤집어쓰며 손으로 고정해 사장님에게 책임감을 인정받으며 칭찬받은 경험이 있습니다 둘째 설비를 혼자 관리하는 것이 아닌 여러 인원이 관리하기 때문에 원활한 의사소통을 통해 공통적인 목표 지향이 필요합니다 . 저는 인도공과대학교 연구실 인턴을 할 때 자성유체 droplet 프로젝트를 통해 현지 적응 못 하는 팀원을 위해 노력한 적이 있습니다 . 매일 두 번을 같이 써브웨이를 먹었고 매일 저녁에 같이 현지인들과 축구를 하였습니다 . 이를 통해 저는 의사소통을 위해 상대방의 상태에 대한 인식도 중요하다고 배웠습니다 . 셋째 수많은 종류의 설비와 최첨단 설비에 대응하기 위해서는 입사 후에도 배움의 자세가 필요하다고 생

In [ ]:
tok_ex = tokenizer_gpt3(example)
print(tok_ex)
print(tok_ex.keys())

print(len(tok_ex['input_ids']))
print(len(tok_ex['token_type_ids']))
print(len(tok_ex['attention_mask']))
print(len(tokenizer_gpt3.vocab))

{'input_ids': [2, 19588, 27306, 19953, 8321, 8107, 35612, 22595, 8271, 5730, 2430, 33143, 20366, 14180, 14536, 8010, 6165, 20113, 2016, 16505, 16248, 24907, 13970, 6266, 8736, 8267, 24177, 33789, 19307, 8314, 8051, 39948, 14097, 4897, 6165, 14061, 16863, 18194, 29376, 8112, 15941, 14041, 2016, 16512, 15252, 15849, 14057, 28949, 8273, 14621, 19192, 14167, 16314, 14604, 20839, 14701, 28949, 8008, 14083, 16690, 34728, 16826, 23160, 18388, 8014, 16168, 28949, 28735, 16487, 3319, 14236, 2016, 15057, 8019, 18545, 16830, 4411, 8034, 31689, 8018, 14258, 15057, 15465, 8289, 14057, 2783, 14170, 23422, 8205, 34050, 29646, 23257, 15302, 14533, 8433, 31826, 19868, 3408, 15666, 15996, 8289, 22319, 28131, 17237, 15996, 3408, 14276, 23810, 8008, 28898, 16321, 14579, 8519, 17763, 14352, 14178, 15864, 14168, 33672, 8034, 14746, 8144, 8017, 15857, 17237, 15996, 15666, 15996, 16634, 14168, 33017, 14790, 3258, 20745, 15882, 24889, 15489, 17101, 15237, 8008, 16321, 14579, 8519, 17763, 14352, 14041, 2016, 14

In [ ]:
for i in tok_ex['input_ids']:
  print(tokenizer_gpt3.convert_ids_to_tokens(i)) #token to id

[CLS]
본인이
회사를
선택할
##때
##의
기준은
무엇이
##며
왜
그
기준에
적합한
##지를
기술
##해
주
##십시오
.
건설
##기술
##연구원
##에서
지
##열
##원
히트
##펌프
co
##p
##을
올리기
위해
삼
주
정도
기계
##실에서
상주
##한
적이
있습니다
.
시설
##관리
##자와
함께
설비
##를
유지
보수
##하면서
책임
##감을
느끼고
자신의
설비
##에
대한
문제를
해결하는
것에
매력을
느꼈
##고
건물
설비
관리에
관심을
두
##었습니다
.
회사
##란
단순히
돈을
버
##는
수단
##이
아닌
회사
성장
##과
함께
나
자신
자아
##실
##천의
장이
되어야
한다고
완성
##차
철강
공장
등
산업
##시설
##과
상업
##빌딩
의료
##시설
등
일반
##건축
##에
이르기까지
넓은
사업
##영
##역을
가지고
있을
뿐만
아니라
해외에서
##는
개발
##도
##상
##국의
의료
##시설
산업
##시설
##뿐만
아니라
동남아
##시아
동
##유럽
등에
수요가
예상
##되며
해외
##에
넓은
사업
##영
##역을
가지고
있습니다
.
저는
정
##적인
자산
##설비
엔지니
##어가
되기
##보단
넓은
영역
##에서
각각의
설비
##에
능
##통
##하고
최첨단
설비
##에
적용하여
배우
##며
폭
##넓
##고
높은
지식을
갖고
넓은
분야에서
성과를
올리는
동
##적인
자산
##설비
엔지니
##어가
되고
싶습니다
.
저는
세
가지
강점
##이
있습니다
##첫
##째
자산
##설비
엔지니
##어는
자신이
담당
##한
설비
##에
대한
책임
##감이
있어야
한다고
생각합니다
.
저는
b
##m
##t
##k
##ore
##a
현장
##실
##습을
할
때
스티
##어링
##펌프
##개
##조
##공사
##중
한
챔
##버
##에
달린
두
개의
유
##압
##펌프
압력
차로
인해
오일이
누
##출이
발생
##하였고
더
큰
피해를
예방
##하고자
오일을
뒤집어
##쓰
##며
손으로
고정
##해
사장님
##에게
책임
##감을
인정
##받
##

In [ ]:
def get_key(val):
    for key, value in tokenizer_gpt3.vocab.items():
         if val == value:
             return key
             
# 예시로 몇 문장 추출해보기
print(get_key(42000))
print(get_key(14567))
print(get_key(2))
print(get_key(20958))

<|startoftext|>
새로운
[CLS]
br


In [ ]:
len(tokenizer_gpt3.vocab) # GPT3 tokenizer 전체 단어수

42003

In [ ]:
# Pytorch 데이터셋 설정
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)
from transformers import GPT2LMHeadModel, GPT2Config
from transformers import AdamW, get_linear_schedule_with_warmup

class GPT2Dataset(Dataset):
  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list: # 자기소개서 별로 encode한 뒤에 tensor로 만들어서 배열에 넣음
      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', 
                                 truncation=True, max_length=max_length, 
                                 padding="max_length") 
      self.input_ids.append(torch.tensor(encodings_dict['input_ids'][1:])) # CLS 토큰 제거
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask'][1:])) # CLS 토큰 제거
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
# 리스트 형태로 변환
coverletter_list = full_data['content']
coverletter_list 

0        본인이 회사를 선택할때의 기준은 무엇이며 왜 그 기준에 적합한지를 기술해 주십시오 .   건설기술연구원에서 지열원 히트펌프 COP을 올리기 위해 삼 주 정도 기계실에서 상주한 적이 있습니다 . 시설관리자와 함께 설비를 유지 보수하면서 책임감을 느끼고 자신의 설비에 대한 문제를 해결하는 것에 매력을 느꼈고 건물 설비 관리에 관심을 두었습니다.회사란 단순히 돈을 버는 수단이 아닌 회사 성장과 함께 나 자신 자아실천의 장이 되어야 한다고 완성차 철강 공장 등 산업시설과 상업빌딩 의료시설 등 일반건축에 이르기까지 넓은 사업영역을 가지고 있을 뿐만 아니라 해외에서는 개발도상국의 의료시설 산업시설뿐만 아니라 동남아시아 동유럽 등에 수요가 예상되며 해외에 넓은 사업영역을 가지고 있습니다.저는 정적인 자산설비 엔지니어가 되기보단 넓은 영역에서 각각의 설비에 능통하고 최첨단 설비에 적용하여 배우며 폭넓고 높은 지식을 갖고 넓은 분야에서 성과를 올리는 동적인 자산설비 엔지니어가 되고 싶습니다.저는 세 가지 강점이 있습니다첫째 자산설비 엔지니어는 자신이 담당한 설비에 대한 책임감이 있어야 한다고 생각합니다 . 저는 BMTKOREA 현장실습을 할 때 스티어링펌프개조공사중 한 챔버에 달린 두 개의 유압펌프 압력 차로 인해 오일이 누출이 발생하였고 더 큰 피해를 예방하고자 오일을 뒤집어쓰며 손으로 고정해 사장님에게 책임감을 인정받으며 칭찬받은 경험이 있습니다 둘째 설비를 혼자 관리하는 것이 아닌 여러 인원이 관리하기 때문에 원활한 의사소통을 통해 공통적인 목표 지향이 필요합니다 . 저는 인도공과대학교 연구실 인턴을 할 때 자성유체 droplet 프로젝트를 통해 현지 적응 못 하는 팀원을 위해 노력한 적이 있습니다 . 매일 두 번을 같이 써브웨이를 먹었고 매일 저녁에 같이 현지인들과 축구를 하였습니다 . 이를 통해 저는 의사소통을 위해 상대방의 상태에 대한 인식도 중요하다고 배웠습니다 . 셋째 수많은 종류의 설비와 최첨단 설비에 대응하기 위해서는 입사 후에도 배움의 자세

In [ ]:
# Pytorch 데이터셋 생성
dataset = GPT2Dataset(coverletter_list, tokenizer_gpt3, max_length=768)

# 훈련/검증 데이터셋으로 분리
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

13,187 training samples
1,466 validation samples


* RandomSampler : <br>
원소를 랜덤하게 샘플링 진행 (중복없이 샘플링하는 경우에 shuffled dataset으로부터 샘플링 진행, 중복해서 샘플링하는 경우에 num_samples 지정)

* SequentialSampler : <br>원소를 순차적으로 샘플링 진행, 항상 같은 순서

In [ ]:
# Pytorch 데이터 로더
batch_size = 2 # GPT모델은 큰 모델이므로 batch_size를 2 초과시 메모리 용량 문제 발생

# Dataloaders for 훈련/검증 데이터셋
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), # batch 랜덤으로 선택
            batch_size = batch_size # 선택된 batch_size로 훈련
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size # 선택된 batch_size로 검증
        )

### 4) 합격 자기소개서 fine-tuning
* GPT2Config : <br>
GPT2 모델의 configuration을 저장하기 위한 용도 (기반(pretrainedconfig)이 되는 GPT-2 모델과 유사한 configuration을 보여줄 것임)T

* GPT2LMHeadModel : <br>
GPT2 트랜스포머 모델과 가장 윗단에 언어 모델(input 임베딩과 연결된 파트)이 있는 모델 구조

In [ ]:
configuration = GPT2Config.from_pretrained('kykim/gpt3-kor-small_based_on_gpt2', output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
import random

model.resize_token_embeddings(len(tokenizer_gpt3)) # token 사이즈 조정

seed_val = 425
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# 하이퍼파라미터 튜닝 (일단은 작게 시작, 나중에 epochs/sample_every 늘리기)
epochs = 3
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
sample_every = 1000

In [ ]:
# AdamW: huggingface library에서의 클래스
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                  )

In [ ]:
# training steps = (number of batches) * (number of epochs)
total_steps = len(train_dataloader) * epochs

# learning rate scheduler (training loop이 지날수록 learning rate 달라짐)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps
                                            )

In [ ]:
import time
import datetime

def format_time(elapsed):
  return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
# 런타임 유형 GPU로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

하이퍼파라미터 조합
- epochs: 3~5
- learning rate: 5e-4
- batch_size: 2
- warmup_steps: 1e2
- epsilon: 1e-8

In [ ]:
total_t0 = time.time()
training_stats = []
model = model.to(device)

# 1) Training
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training Start')

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device) # GPU 입력으로 사용될 tensor는 모두 to(device) 필요
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()      # 변화도(Gradient) 매개변수를 0으로 만들고   

        outputs = model(b_input_ids, # loss를 출력하는지 확인
                        labels=b_labels, 
                        attention_mask = b_masks,
                        token_type_ids=None 
                        )

        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # x batch마다 샘플 불러오기
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),  # beginning sequence token의 id (랜덤한 단어로 시작하계끔)
                                    do_sample=True,  # sampling 사용할지 여부 판단 (use greedy decoding)  
                                    top_k=50, # number of highest probability vocab tokens
                                    max_length = 200, # maximum length of the sequence to be generated
                                    top_p=0.95,
                                    num_return_sequences=1, # 한 배치당 returned sequence의 수
                                    repetition_penalty=2.0,     
                                    )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer_gpt3.decode(sample_output, skip_special_tokens=True)))
            
            model.train() # train은 일정 sample_every step마다

        loss.backward()
        optimizer.step()
        scheduler.step()

    # 모든 batch에 대해 평균 loss 계산
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # epoch별로 소요시간 측정
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    # 2) Validation
    print("")
    print("Validation Start")

    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
                            #token_type_ids=None, 
                            attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training Complete")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
# epoch별로 train loss, val loss, train time, val time 표시
pd.set_option('precision', 2)
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch') # index = epoch
df_stats

In [ ]:
sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (10,8)

plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])
plt.show()

In [ ]:
# 파라미터 표시 & transformer의 층 구조
params = list(model.named_parameters())
print('GPT-2 모델은 {:} 개의 다른 파라미터를 갖습니다.\n'.format(len(params)))

print('==== Embedding Layer ====\n')
for p in params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')
for p in params[2:14]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')
for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

GPT-2 모델은 148 개의 다른 파라미터를 갖습니다.

==== Embedding Layer ====

transformer.wte.weight                                  (42004, 768)
transformer.wpe.weight                                   (2048, 768)

==== First Transformer ====

transformer.h.0.ln_1.weight                                   (768,)
transformer.h.0.ln_1.bias                                     (768,)
transformer.h.0.attn.c_attn.weight                       (768, 2304)
transformer.h.0.attn.c_attn.bias                             (2304,)
transformer.h.0.attn.c_proj.weight                        (768, 768)
transformer.h.0.attn.c_proj.bias                              (768,)
transformer.h.0.ln_2.weight                                   (768,)
transformer.h.0.ln_2.bias                                     (768,)
transformer.h.0.mlp.c_fc.weight                          (768, 3072)
transformer.h.0.mlp.c_fc.bias                                (3072,)
transformer.h.0.mlp.c_proj.weight                        (3072, 768)
transformer.h

In [ ]:
# 모델 저장
output_dir = '/content/drive/MyDrive/model_save'

# 디렉토리 만들기
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print("Saving model to %s" % output_dir)

# 모델, 토크나이저 등 저장
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer_gpt3.save_pretrained(output_dir)

Saving model to /content/drive/MyDrive/model_save


('/content/drive/MyDrive/model_save/tokenizer_config.json',
 '/content/drive/MyDrive/model_save/special_tokens_map.json',
 '/content/drive/MyDrive/model_save/vocab.txt',
 '/content/drive/MyDrive/model_save/added_tokens.json',
 '/content/drive/MyDrive/model_save/tokenizer.json')

In [ ]:
!ls -l --block-size=K /content/drive/MyDrive/model_save # 저장 잘 됐나 확인

total 514610K
-rw------- 1 root root      1K Aug 29 13:36 added_tokens.json
-rw------- 1 root root      1K Aug 29 13:36 config.json
-rw------- 1 root root 513616K Aug 29 13:36 pytorch_model.bin
-rw------- 1 root root      1K Aug 29 13:36 special_tokens_map.json
-rw------- 1 root root      1K Aug 29 13:36 tokenizer_config.json
-rw------- 1 root root    656K Aug 29 13:36 tokenizer.json
-rw------- 1 root root    337K Aug 29 13:36 vocab.txt


In [ ]:
!ls -l --block-size=M /content/drive/MyDrive/model_save/pytorch_model.bin # pytorch model 저장 잘 됐나 확인

### 5) 텍스트 생성

In [ ]:
# 구글 드라이브에서 모델, 토크나이저 불러오기
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/model_save')

from transformers import BertTokenizerFast, GPT2LMHeadModel
tokenizer = BertTokenizerFast.from_pretrained('/content/drive/MyDrive/model_save')
input_ids = tokenizer.encode("text to tokenize")[1:]  # CLS token 제거

In [ ]:
device = torch.device("cuda")
model.cuda()

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model = model.to(device)

In [ ]:
model.eval()

def coverletter_generator(text):
  prompt = f"<|startoftext|> {text}"
  generated = torch.tensor(tokenizer_gpt3.encode(prompt)[1:]).unsqueeze(0)
  generated = generated.to(device)

  print(generated)

  sample_outputs = model.generate(
                                  generated, 
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 300,
                                  top_p=0.95, 
                                  num_return_sequences=3,
                                  repetition_penalty=1.1
                                  )

  for i, sample_output in enumerate(sample_outputs):
    result = "{}: {}\n\n".format(i, tokenizer_gpt3.decode(sample_output, skip_special_tokens=True))
    print(result)
  return